# KB Mirrors

This example simulates the alignment of a KB mirror endstation (with four degrees of freedom).

In [ ]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
from bluesky.callbacks import best_effort
from bluesky.callbacks.tiled_writer import TiledWriter
from bluesky.run_engine import RunEngine
from ophyd.utils import make_dir_tree
from tiled.client import from_uri
from tiled.server import SimpleTiledServer
from blop.sim import beamline

# Initialize Tiled server and client
tiled_server = SimpleTiledServer(readable_storage=[beamline.DECTECTOR_STORAGE])
tiled_client = from_uri(tiled_server.uri)
tiled_writer = TiledWriter(tiled_client)


# Set up RE enviornment
def setup_re_env(db_type="default", root_dir="/default/path"):
    RE = RunEngine({})
    bec = best_effort.BestEffortCallback()
    RE.subscribe(bec)
    RE.subscribe(tiled_writer)
    _ = make_dir_tree(datetime.datetime.now().year, base_path=root_dir)
    return {"RE": RE, "tiled": tiled_client, "bec": bec} 

env = setup_re_env(db_type="temp", root_dir="/tmp/blop/test/sim")
globals().update(env)

bec.disable_plots()

/home/jmoylan1/miniconda3/envs/test_tiled_new/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-04 11:20:08.985 INFO: Subprocess stdout: 
2025-06-04 11:20:08.986 INFO: Subprocess stderr: 
Tiled version 0.1.dev2763+ge244fae
2025-06-04 11:20:09.279 INFO: Tiled version 0.1.dev2763+ge244fae
2025-06-04 11:20:09.282 INFO: Context impl SQLiteImpl.
2025-06-04 11:20:09.282 INFO: Will assume non-transactional DDL.
2025-06-04 11:20:09.387 INFO: HTTP Request: GET http://127.0.0.1:35787/api/v1?api_key=75d5f7c81cebfc7a "HTTP/1.1 307 Temporary Redirect"
2025-06-04 11:20:09.393 INFO: HTTP Request: GET http://127.0.0.1:35787/api/v1/ "HTTP/1.1 200 OK"
2025-06-04 11:20:09.415 INFO: HTTP Request: GET http://127.0.0.1:35787/api/v1/metadata/ "HTTP/1.1 200 OK"


2025-06-04 11:20:13.114 INFO: HTTP Request: GET http://127.0.0.1:35787/api/v1/table/partition/f7763365-cc6e-4b5e-a43d-3cfe6843c596/primary/internal/events?partition=0 "HTTP/1.1 200 OK"
2025-06-04 11:20:13.544 INFO: HTTP Request: GET http://127.0.0.1:35787/api/v1/array/block/f7763365-cc6e-4b5e-a43d-3cfe6843c596/primary/external/bl_det_image?block=4%2C0%2C0&expected_shape=1%2C300%2C400 "HTTP/1.1 500 Internal Server Error"
2025-06-04 11:20:13.546 WARNING: Scheduled retry in 0.32 seconds due to HTTPStatusError("Server error '500 Internal Server Error' for url 'http://127.0.0.1:35787/api/v1/array/block/f7763365-cc6e-4b5e-a43d-3cfe6843c596/primary/external/bl_det_image?block=4%2C0%2C0&expected_shape=1%2C300%2C400'\nFor more information, server admin can search server logs for correlation ID eb7b5bf2f9ae4fa6.") (attempt 1)
2025-06-04 11:20:13.553 INFO: HTTP Request: GET http://127.0.0.1:35787/api/v1/array/block/f7763365-cc6e-4b5e-a43d-3cfe6843c596/primary/external/bl_det_image?block=29%2C0%2C

In [2]:
tiled_server.directory

PosixPath('/tmp/blop/sim')

In [3]:
from blop.sim import Beamline

beamline = Beamline(name="bl")

In [ ]:
from blop import DOF, Agent, Objective
from blop.digestion import beam_stats_digestion

dofs = [
    DOF(description="KBV downstream", device=beamline.kbv_dsv, search_domain=(-5.0, 5.0)),
    DOF(description="KBV upstream", device=beamline.kbv_usv, search_domain=(-5.0, 5.0)),
    DOF(description="KBH downstream", device=beamline.kbh_dsh, search_domain=(-5.0, 5.0)),
    DOF(description="KBH upstream", device=beamline.kbh_ush, search_domain=(-5.0, 5.0)),
]

objectives = [
    Objective(name="bl_det_sum", target="max", transform="log", trust_domain=(200, np.inf)),
    Objective(name="bl_det_wid_x", target="min", transform="log", latent_groups=[("bl_kbh_dsh", "bl_kbh_ush")]),
    Objective(name="bl_det_wid_y", target="min", transform="log", latent_groups=[("bl_kbv_dsv", "bl_kbv_usv")]),
]

agent = Agent(
    dofs=dofs,
    objectives=objectives,
    detectors=[beamline.det],
    digestion=beam_stats_digestion,
    digestion_kwargs={"image_key": "bl_det_image"},
    verbose=True,
    tiled=tiled,
    tolerate_acquisition_errors=False,
    enforce_all_objectives_valid=True,
    train_every=3,
)
(uid,) = RE(agent.learn("qr", n=32))

2025-06-04 11:20:09.431 INFO: Executing plan <generator object Agent.learn at 0x736f6e19b240>
2025-06-04 11:20:09.431 INFO: Change state on <bluesky.run_engine.RunEngine object at 0x736db2783750> from 'idle' -> 'running'
2025-06-04 11:20:09.432 INFO: running iteration 1 / 1




Transient Scan ID: 1     Time: 2025-06-04 11:20:09
Persistent Unique Scan ID: 'f7763365-cc6e-4b5e-a43d-3cfe6843c596'


2025-06-04 11:20:09.474 INFO: HTTP Request: POST http://127.0.0.1:35787/api/v1/metadata/ "HTTP/1.1 200 OK"


New stream: 'primary'
+-----------+------------+------------+------------+------------+------------+------------+--------------+--------------+--------------+--------------+
|   seq_num |       time | bl_kbv_dsv | bl_kbv_usv | bl_kbh_dsh | bl_kbh_ush | bl_det_sum | bl_det_cen_x | bl_det_cen_y | bl_det_wid_x | bl_det_wid_y |
+-----------+------------+------------+------------+------------+------------+------------+--------------+--------------+--------------+--------------+


2025-06-04 11:20:09.520 INFO: HTTP Request: GET http://127.0.0.1:35787/api/v1/search//f7763365-cc6e-4b5e-a43d-3cfe6843c596?page%5Boffset%5D=0&fields=&sort= "HTTP/1.1 200 OK"
2025-06-04 11:20:09.532 INFO: HTTP Request: POST http://127.0.0.1:35787/api/v1/metadata//f7763365-cc6e-4b5e-a43d-3cfe6843c596 "HTTP/1.1 200 OK"
2025-06-04 11:20:09.542 INFO: HTTP Request: POST http://127.0.0.1:35787/api/v1/metadata//f7763365-cc6e-4b5e-a43d-3cfe6843c596/primary "HTTP/1.1 200 OK"
2025-06-04 11:20:09.552 INFO: HTTP Request: POST http://127.0.0.1:35787/api/v1/metadata//f7763365-cc6e-4b5e-a43d-3cfe6843c596/primary "HTTP/1.1 200 OK"
2025-06-04 11:20:09.561 INFO: HTTP Request: POST http://127.0.0.1:35787/api/v1/metadata//f7763365-cc6e-4b5e-a43d-3cfe6843c596/primary "HTTP/1.1 200 OK"
2025-06-04 11:20:09.574 INFO: HTTP Request: PATCH http://127.0.0.1:35787/api/v1/metadata//f7763365-cc6e-4b5e-a43d-3cfe6843c596/primary "HTTP/1.1 200 OK"
2025-06-04 11:20:09.581 INFO: HTTP Request: GET http://127.0.0.1:35787/ap

...

|        32 | 11:20:12.9 |      3.819 |      4.528 |     -3.252 |      4.583 |     -4.436 |      199.405 |      155.944 |      381.018 |      210.639 |


2025-06-04 11:20:12.921 INFO: HTTP Request: GET http://127.0.0.1:35787/api/v1/metadata//f7763365-cc6e-4b5e-a43d-3cfe6843c596/primary/internal "HTTP/1.1 200 OK"
2025-06-04 11:20:12.931 INFO: HTTP Request: GET http://127.0.0.1:35787/api/v1/search/f7763365-cc6e-4b5e-a43d-3cfe6843c596/primary/internal?page%5Boffset%5D=0&fields=&sort= "HTTP/1.1 200 OK"
2025-06-04 11:20:12.941 INFO: HTTP Request: GET http://127.0.0.1:35787/api/v1/metadata/f7763365-cc6e-4b5e-a43d-3cfe6843c596/primary/internal/events "HTTP/1.1 200 OK"
2025-06-04 11:20:12.953 INFO: HTTP Request: PATCH http://127.0.0.1:35787/api/v1/table/partition/f7763365-cc6e-4b5e-a43d-3cfe6843c596/primary/internal/events?partition=0 "HTTP/1.1 200 OK"


+-----------+------------+------------+------------+------------+------------+------------+--------------+--------------+--------------+--------------+
generator list_scan ['f7763365'] (scan num: 1)





2025-06-04 11:20:13.067 INFO: HTTP Request: PATCH http://127.0.0.1:35787/api/v1/metadata//f7763365-cc6e-4b5e-a43d-3cfe6843c596 "HTTP/1.1 200 OK"
2025-06-04 11:20:13.080 INFO: HTTP Request: GET http://127.0.0.1:35787/api/v1/metadata/f7763365-cc6e-4b5e-a43d-3cfe6843c596 "HTTP/1.1 200 OK"
2025-06-04 11:20:13.092 INFO: HTTP Request: GET http://127.0.0.1:35787/api/v1/metadata/f7763365-cc6e-4b5e-a43d-3cfe6843c596/primary/internal/events "HTTP/1.1 200 OK"
2025-06-04 11:20:13.125 INFO: HTTP Request: GET http://127.0.0.1:35787/api/v1/metadata/f7763365-cc6e-4b5e-a43d-3cfe6843c596 "HTTP/1.1 200 OK"
2025-06-04 11:20:13.140 INFO: HTTP Request: GET http://127.0.0.1:35787/api/v1/metadata/f7763365-cc6e-4b5e-a43d-3cfe6843c596/primary/external/bl_det_image "HTTP/1.1 200 OK"


A 'deferred pause' has been requested. The RunEngine will pause at the next checkpoint. To pause immediately, hit Ctrl+C again in the next 10 seconds.


In [ ]:
RE(agent.learn("qei", n=4, iterations=4))

In [ ]:
plt.imshow(agent.best.bl_det_image)

In [ ]:
agent.plot_objectives(axes=(2, 3))

In [ ]:
tiled_server.close()